##
## APROXIMACION INGENUA
##

#
# Empaquetado del entrenamiento del modelo
#

In [4]:
# Elastic net 


def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):
    #verbose imprime con mas o menor detalle
    #entrena modelos y me deja escoger el de menor error

    import os
    import pickle

    import pandas as pd
    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from sklearn.model_selection import train_test_split

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=0,
    )

    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)
    #coeficientes elevados al cuadrado y el valor absoluto, se enfocan en mejorar el sobre ajuste, semilla fija en random state

    estimator.fit(x_train, y_train)
    y_pred = estimator.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    if verbose > 0:
        print(estimator, ":", sep="")
        print(f"  MSE: {mse}")
        print(f"  MAE: {mae}")
        print(f"  R2: {r2}")

    if not os.path.exists("estimator.pickle"):
        saved_estimator = None
    else:
        with open("estimator.pickle", "rb") as file:
            saved_estimator = pickle.load(file)

    if saved_estimator is None or estimator.score(
        x_test, y_test
    ) > saved_estimator.score(x_test, y_test):
        with open("estimator.pickle", "wb") as file:
            pickle.dump(estimator, file)
    #guarda el modelo si es mejor, y solo lo guarda si es mejor

#
# Experimiento
#

In [5]:
train_estimator(0.5, 0.5)

ElasticNet(alpha=0.5, random_state=12345):
  MSE: 0.5294843132862007
  MAE: 0.5894666734018875
  R2: 0.13368827268570616


In [6]:
train_estimator(0.2, 0.2)

ElasticNet(alpha=0.2, l1_ratio=0.2, random_state=12345):
  MSE: 0.43869119518947153
  MAE: 0.5236106762028768
  R2: 0.2822387414965034


# _________________________ Refactorizacion del codigo --------------------------------

#
# Carga de datos
#

In [7]:
def load_data():

    import pandas as pd

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    return x, y
  

#
# Particionamiento de datos
#

In [8]:
def make_train_test_split(x, y):

    from sklearn.model_selection import train_test_split

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=0,
    )
    return x_train, x_test, y_train, y_test
  
  

#
# Cálculo de metricas de evaluación
#

In [9]:
def eval_metrics(y_true, y_pred):

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2

#
# Reporte de métricas de evaluación
#

In [10]:
def report(estimator, mse, mae, r2):

    print(estimator, ":", sep="")
    print(f"  MSE: {mse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")
    

#
# Almacenamiento del modelo
#

In [11]:
def save_best_estimator(estimator):

    import os
    import pickle

    with open("estimator.pickle", "wb") as file:
        pickle.dump(estimator, file)
        

#
# Carga del modelo
#

In [12]:
def load_best_estimator():

    import os
    import pickle

    if not os.path.exists("estimator.pickle"):
        return None
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)

    return estimator

#
# Entrenamiento
#

In [13]:
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):

    from sklearn.linear_model import ElasticNet

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)
    estimator.fit(x_train, y_train)
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    if verbose > 0:
        report(estimator, mse, mae, r2)

    best_estimator = load_best_estimator()
    if best_estimator is None or estimator.score(x_test, y_test) > best_estimator.score(
        x_test, y_test
    ):
        best_estimator = estimator

    save_best_estimator(best_estimator)

In [14]:
train_estimator(0.5, 0.5)

ElasticNet(alpha=0.5, random_state=12345):
  MSE: 0.5294843132862007
  MAE: 0.5894666734018875
  R2: 0.13368827268570616


In [15]:
train_estimator(0.2, 0.2)

ElasticNet(alpha=0.2, l1_ratio=0.2, random_state=12345):
  MSE: 0.43869119518947153
  MAE: 0.5236106762028768
  R2: 0.2822387414965034


In [16]:
train_estimator(0.1, 0.1)

ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=12345):
  MSE: 0.4183271587407731
  MAE: 0.5055024368693067
  R2: 0.31555720466583137


In [17]:
train_estimator(0.05, 0.05)

ElasticNet(alpha=0.05, l1_ratio=0.05, random_state=12345):
  MSE: 0.40816015194032085
  MAE: 0.49587028530073707
  R2: 0.33219187542359396


#
# Chekeador de estimador
#

In [18]:
#no es usada en la practica
def check_estimator():

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    estimator = load_best_estimator()
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    report(estimator, mse, mae, r2)


#
# Debe coincidir con el mejor modelo encontrado en la celdas anteriores
#
check_estimator()

ElasticNet(alpha=0.05, l1_ratio=0.05, random_state=12345):
  MSE: 0.40816015194032085
  MAE: 0.49587028530073707
  R2: 0.33219187542359396


 ___________ BUSQUEDA POR CODIGO _______________

In [19]:
def make_hyperparameters_search(alphas, l1_ratios):
    
    for alpha in alphas:
        for l1_ratio in l1_ratios:
            train_estimator(alpha, l1_ratio=l1_ratio, verbose=0)

#
# Entrenando con numpy
#

In [20]:
import numpy as np

alphas = np.linspace(0.0001, 0.5, 10)
l1_ratios = np.linspace(0.0001, 0.5, 10)
make_hyperparameters_search(alphas, l1_ratios)
check_estimator()

ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785


######
### __________________- USANDO GRIDSEARCH ____________________

In [21]:
def train_estimator(alphas, l1_ratios, n_splits=5, verbose=1):

    from sklearn.linear_model import ElasticNet
    from sklearn.model_selection import GridSearchCV

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)

    # -------------------------------------------------------------------------
    # Búsqueda de parámetros con validación cruzada
    #
    estimator = GridSearchCV(
        estimator=ElasticNet(
            random_state=12345,
        ),
        param_grid={
            "alpha": alphas,
            "l1_ratio": l1_ratios,
        },
        cv=n_splits,
        refit=True,
        verbose=0,
        return_train_score=False,
    )
    # -------------------------------------------------------------------------
    #EL solo combina todos los posibles valores de alpha y l1

    estimator.fit(x_train, y_train)

    estimator = estimator.best_estimator_

    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    if verbose > 0:
        report(estimator, mse, mae, r2)

    best_estimator = load_best_estimator()
    if best_estimator is None or estimator.score(x_test, y_test) > best_estimator.score(
        x_test, y_test
    ):
        best_estimator = estimator

    save_best_estimator(best_estimator)


In [22]:
import numpy as np

train_estimator(
    alphas=np.linspace(0.0001, 0.5, 10),
    l1_ratios=np.linspace(0.0001, 0.5, 10),
    n_splits=5,
    verbose=1,
)

ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785
